# Hack The Feed: Insights From Social Media Data

### 🎯 Project Brief

Playhouse Communication is one of Nigeria's leading digital marketing agencies. They combine design and media planning with cutting-edge tech solutions to reimagine what marketing is all about. Their client roster is a mix of global juggernauts and nimble SMEs, each redefining their sectors.

We are rolling out the ultimate arena for innovation in data and setting the stage for up and coming data scientists and analysts to showcase their skills, win huge cash prizes, and boost their careers. The "Hack the Feed" hackathon is a showdown where data analytics meets creative prowess.

Your mission? To decode a treasure trove of social media data for one of our high-profile clients and transform it into game-changing insights.

In a rare move, we're handing you the keys to a vault of exclusive social media data to let you dig deep, get creative, and strike gold with actionable insights that could redefine the future of digital marketing.  This isn't just a hackathon; it's your chance to shape the future of digital engagement. 🚀

Key Deliverables:
Participants are expected to:

    Create a comprehensive and reproducible report detailing their findings.
    Propose actionable recommendations based on the insights.
    Create a simple and engaging visualisation of your results & analysis.


Evaluation Criteria:
Submissions will be evaluated based on the following:

    Innovativeness:
        Originality and novelty of the insights.
    Actionability:
        Practicality and feasibility of the recommendations.
    Presentation Quality:
        Clarity and effectiveness in conveying findings in writing and visual form.
    Depth of Analysis:
        How thoroughly the data has been explored and understood.

In [ ]:
import os
import gc
import shutil
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import random
from collections import defaultdict
from typing import Union, List, Literal, Dict, Callable, Tuple, Optional
from tqdm import tqdm

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, KFold

In [ ]:
data_dir = '/content/drive/MyDrive/Hack The Feed Hackathon'
os.chdir(data_dir)

In [ ]:
fb_data = pd.read_csv('Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Facebook.csv', low_memory=False)
x_data = pd.read_csv('Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Twitter.csv', low_memory=False)
ig_data = pd.read_csv('Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_Instagram.csv', low_memory=False)
ld_data = pd.read_csv('Post Performance (Stanbic IBTC) January 1, 2013 - July 13, 2023_LinkedIn.csv', low_memory=False)

## Facebook_Data

In [ ]:
fb_data.info()

# 78 Numerical Columns
# 69 Object Columns

In [ ]:
low_cardinality_columns = []
high_cardinality_columns = []
medium_cardinality_columns = []
cat_cols = fb_data.select_dtypes(include=['object']).columns

for col in cat_cols:
  num_uniq = fb_data[col].nunique()
  if num_uniq < 5:
    low_cardinality_columns.append(col)
  elif num_uniq > 20:
    high_cardinality_columns.append(col)
  else:
    medium_cardinality_columns.append(col)

In [ ]:
fb_data[low_cardinality_columns].head(3)

### HIGH CARDINALITY COLUMNS

In [ ]:
fb_data[high_cardinality_columns[:5]].head(5)

In [ ]:
impression_cols = high_cardinality_columns[5:13]
reach_cols = high_cardinality_columns[13:18]
fb_data[impression_cols].head(5)

In [ ]:
fb_data[reach_cols].head(5)

In [ ]:
fb_data[impression_cols].isnull().sum()

In [ ]:
fb_data[reach_cols].isnull().sum()

In [ ]:
cleaned_impressions_reach = fb_data

for col in cleaned_impressions_reach[impression_cols].columns:
  cleaned_impressions_reach = cleaned_impressions_reach[cleaned_impressions_reach[col].notna()]

for col in cleaned_impressions_reach[reach_cols].columns:
  cleaned_impressions_reach = cleaned_impressions_reach[cleaned_impressions_reach[col].notna()]

In [ ]:
cleaned_impressions_reach[reach_cols] = cleaned_impressions_reach[reach_cols].apply(
    lambda row:row.str.replace(',','').astype("int"), axis=1)

In [ ]:
cleaned_impressions_reach[impression_cols] = cleaned_impressions_reach[impression_cols].apply(
    lambda row:row.str.replace(',','').astype("int"), axis=1)

In [ ]:
def get_time_period(hour):
    if 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

def get_quarter(month):
    if 1 <= month <= 3:
        return 'Q1'
    elif 4 <= month <= 6:
        return 'Q2'
    elif 7 <= month <= 9:
        return 'Q3'
    else:
        return 'Q4'

def get_season(month):
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

def get_day_period(day_of_week):
    if 0 <= day_of_week <= 4:  # Monday to Friday
        return 'Weekday'
    else:  # Saturday and Sunday
        return 'Weekend'

In [ ]:
holidays = pd.read_excel('NigerianHolidays.xlsx')
holidays['month-day'] = pd.to_datetime(holidays.Date).dt.strftime('%m-%d')

In [ ]:
holidays = holidays.set_index('month-day')
holiday_names = holidays['Name'].to_dict()
holiday_types = holidays['Type'].to_dict()

In [ ]:
cleaned_impressions_reach['year'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.year
cleaned_impressions_reach['month_name'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.month_name()
cleaned_impressions_reach['month'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.month
cleaned_impressions_reach['day_name'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.day_name()
cleaned_impressions_reach['day'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.day
cleaned_impressions_reach['hour'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.hour
cleaned_impressions_reach['minute'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.minute
cleaned_impressions_reach['day_of_week'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.dayofweek
cleaned_impressions_reach['month-day'] = pd.to_datetime(cleaned_impressions_reach.Date).dt.strftime('%m-%d')

In [ ]:
cleaned_impressions_reach['time_period'] = cleaned_impressions_reach['hour'].apply(get_time_period)
cleaned_impressions_reach['quarter'] = cleaned_impressions_reach['month'].apply(get_quarter)
cleaned_impressions_reach['season'] = cleaned_impressions_reach['month'].apply(get_season)
cleaned_impressions_reach['day_period'] = cleaned_impressions_reach['day_of_week'].apply(get_day_period)
cleaned_impressions_reach['holiday_names'] = cleaned_impressions_reach['month-day'].map(holiday_names).fillna("Regular Day")
cleaned_impressions_reach['holiday_types'] = cleaned_impressions_reach['month-day'].map(holiday_types).fillna("Regular Type")

In [ ]:
order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday','Saturday', 'Sunday']
cleaned_impressions_reach['day_name'] = pd.Categorical(cleaned_impressions_reach['day_name'], categories=order, ordered=True)



order = ['Morning', 'Afternoon', 'Evening', 'Night']
cleaned_impressions_reach['time_period'] = pd.Categorical(cleaned_impressions_reach['time_period'], categories=order, ordered=True)

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  df = cleaned_impressions_reach[cleaned_impressions_reach[col]!='Regular Type']
  df_yearly = df.groupby(col)[['Post']].count()  # Sum column values for each year
  axs[i].plot(df_yearly.index, df_yearly.values, marker='o')

  axs[i].set_xlabel(col)
  axs[i].set_ylabel('Count')
  axs[i].set_title(f'Trend of Posts Across {col}')

plt.show()

In [ ]:
cleaned_impressions_reach[impression_cols].describe().T

In [ ]:
last_quantile = cleaned_impressions_reach['Impressions'].quantile(0.99) #Posts that had Impression in the Top 1%
df = cleaned_impressions_reach[cleaned_impressions_reach['Impressions'] > last_quantile]

df.groupby('year')[impression_cols[:2]].sum()

In [ ]:
last_quantile = cleaned_impressions_reach['Impressions'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Impressions'] > last_quantile]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in impression_cols[:2]:
    df_rt = df[df[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()


There doesn't seem to be a correlation between the number of posts in a year and the number of Impressions gotten across the Year. even when there was a sudden increase in the number of posts in the year 2018 - 2022, there was a large decrease in the sum of Impressions in 2018, and a large decrease between the years 2019 - 2022.

The number of Impressions across the Time period, seems to decrease linearly from Morning till Evening, this may very well be part of the Facebook Algorithm

Thursday and Saturday seem to have the highest number of impressions, compared to wednesday and friday who have the most posts, what makes thursday and saturday so special?

The sum of Impressions rises from the first quarter to the second quarter but drops rapidly till the fourth quarter, there seems to be a lot of unorganic impressions after the 2nd quarter

Christian Holidays and other religious Holidays have more impressions than others

    The Highest Impressions Scores on the Posts were between the years 2018 and 2020, with 2019 having the highest possible values, there was a spike from 2017 which quickly went down but rose up from 2018 till 2019 and then there was a sharp drop from 2019 till 2020.

    Having lowest dips in 2018 and 2022

In [ ]:
last_quantile = cleaned_impressions_reach['Non-viral Impressions'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Non-viral Impressions'] > last_quantile]

df.groupby('year')[impression_cols[2:4]].sum()

In [ ]:
last_quantile = cleaned_impressions_reach['Non-viral Impressions'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Non-viral Impressions'] > last_quantile]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in impression_cols[2:4]:
    df_rt = df[df[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()


    Based on the table, here are some creative insights and patterns:

    Viral Impressions: The highest number of Viral Impressions, which represents the number of times a post was displayed because it was shared by users and their friends, occurred in 2019. However, there was a significant drop in Viral Impressions in the subsequent years, with the lowest in 2022. This could suggest that the content posted in 2019 resonated particularly well with the audience, prompting them to share it with their friends. The sharp decrease in the following years might indicate a change in content strategy or audience behavior that led to less sharing.

    Non-viral Impressions: The highest number of Non-viral Impressions, which represents the number of times any content from your Page entered a user’s screen excluding instances when someone’s friend likes or follows your Page, engages with a post, shares a photo of your Page, and checks into your Page, occurred in 2019. There was a decrease in Non-viral Impressions in 2020 and 2021, but it increased again in 2023. This could indicate that while fewer users were discovering the content organically in 2020 and 2021, the visibility of the content improved again in 2023.

    These trends suggest that while the virality of the content (as measured by Viral Impressions) decreased after 2019, the overall visibility of the content (as measured by Non-viral Impressions) remained relatively high. This could indicate that while fewer users were sharing the content with their friends over time, the content was still reaching a large audience through other means. It’s important to delve deeper into what might have caused these trends to better inform future content strategies.

    For instance, you could explore questions like:

    What were the characteristics of posts from 2019 that drove high Viral and Non-viral Impressions?
    What changes occurred in 2020 and 2021 that might have led to a decrease in Non-viral Impressions?
    What strategies were implemented in 2023 that led to an increase in Non-viral Impressions?

    Viral Impressions had a slight drop from 2017-2018 and rose quickly from 2018-2019 but dropped from 2019-2022 but rose from 2022-2023, same thing can be observed from Non-Viral Impressions, a lot of people stopped interacting with the posts from the periods of 2019-2022, why so?

In [ ]:
last_quantile = cleaned_impressions_reach['Fan Impressions'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Fan Impressions'] > last_quantile]

df.groupby('year')[impression_cols[4:]].sum()

In [ ]:
last_quantile = cleaned_impressions_reach['Fan Impressions'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Fan Impressions'] > last_quantile]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in impression_cols[4:]:
    df_rt = df[df[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

    A sudden decrease from 2017-2018 and then a sharp increan from 2018-2019 and then a sudden decrease from 2019-2022 can be observed from the Fan Impressions, Fan Organic Impressions and Non Fan Impressions, Non Fan Organic Impressions

In [ ]:
cleaned_impressions_reach[reach_cols].describe()

In [ ]:
last_quantile = cleaned_impressions_reach['Reach'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Reach'] > last_quantile]

df.groupby('year')[reach_cols[:2]].sum()

In [ ]:
last_quantile = cleaned_impressions_reach['Reach'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Reach'] > last_quantile]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in reach_cols[:2]:
    df_rt = df[df[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

In [ ]:
last_quantile = cleaned_impressions_reach['Non-viral Reach'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Non-viral Reach'] > last_quantile]

df.groupby('year')[reach_cols[2:4]].sum()

      
    year 		Viral Reach 	Non-viral Reach
    2017 	   63985 	      237526
    2018 	   11830 	       64202
    2019 	  133035 	      829701
    2020 	   63622 	      538092
    2021 	   55165 	      591436
    2022 	   8962 	       556239
    2023 	   88301 	      653503

In [ ]:
last_quantile = cleaned_impressions_reach['Non-viral Reach'].quantile(0.99)
df = cleaned_impressions_reach[cleaned_impressions_reach['Non-viral Reach'] > last_quantile]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in reach_cols[2:4]:
    df_rt = df[df[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()


In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(25,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in impression_cols:
    df_rt = cleaned_impressions_reach[cleaned_impressions_reach[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()


In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(25,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in reach_cols:
    df_rt = cleaned_impressions_reach[cleaned_impressions_reach[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()


In [ ]:
cleaned_impressions_reach[cleaned_impressions_reach['Organic Impressions'] > cleaned_impressions_reach['Impressions']][impression_cols]



> When the number of Impressions equals the number of Organic Impressions, it means that all of the times your content was seen were organic impressions. This means that your content was not promoted in any way, but it was still seen by a large number of people.

> When the number of Organic Impressions is higher than the number of Impressions, it means that you have some viral impressions. Viral impressions are impressions that come from people sharing, liking, or commenting on your content. This is a good sign, because it means that your content is engaging and people are interested in sharing it with their friends.

> When the number of Impressions doesn't equal the number of Organic Impressions, it means that you have some non-organic impressions. Non-organic impressions are impressions that come from promoted content or from people who have seen your content through other means, such as search results or other websites.


    If all of your impressions are organic, it means that your content is performing well on its own. However, if you want to reach more people, you could consider promoting your content.

    If you have some viral impressions, it means that your content is engaging and people are interested in sharing it with their friends. This is a good sign, and you should continue to create high-quality content that is likely to be shared.

    If you have some non-organic impressions, it means that people are seeing your content through other means besides your Facebook Page. This could be a good thing, but it's important to make sure that your content is still relevant and engaging to your target audience.



In [ ]:
cleaned_impressions_reach[cleaned_impressions_reach['Viral Impressions'] > cleaned_impressions_reach['Non-viral Impressions']][impression_cols]

In [ ]:
cleaned_impressions_reach[cleaned_impressions_reach['Organic Reach'] > cleaned_impressions_reach['Reach']][reach_cols]

> When the number of Reach equals the number of Organic Reach, it means that all of the times your content was seen were organic impressions. This means that your content was not promoted in any way, but it was still seen by a large number of people.

> When the number of Organic Reach is higher than the number of Reach, it means that you have some viral impressions. Viral impressions are impressions that come from people sharing, liking, or commenting on your content. This is a good sign, because it means that your content is engaging and people are interested in sharing it with their friends.

> When the number of Reach doesn't equal the number of Organic Reach, it means that you have some non-organic impressions. Non-organic impressions are impressions that come from promoted content or from people who have seen your content through other means, such as search results or other websites.

In [ ]:
cleaned_impressions_reach[cleaned_impressions_reach['Viral Reach'] > cleaned_impressions_reach['Non-viral Reach']][reach_cols]

In [ ]:
engagement_cols = high_cardinality_columns[18:21]
cleaned_impressions_reach['Engagements'] = cleaned_impressions_reach['Engagements'].str.replace(',','').astype('int')
cleaned_impressions_reach[engagement_cols + impression_cols[:2] + reach_cols[:2]].head(5)

Engagement Rate(per Impression) = Engagments / Impressions

Engagement Rate(per Reach) = Engagements/ Reach

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,10))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  df_rt = cleaned_impressions_reach[cleaned_impressions_reach[col]!='Regular Type']
  df_yearly = df_rt.groupby(col)['Engagements'].sum()  # Sum column values for each year
  axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

  axs[i].set_xlabel('Engagements')
  axs[i].set_ylabel('Count')
  axs[i].set_title(f'Trend of Engagements Across {col}')

plt.show()


In [ ]:
def heatmap(df, length=10, width=5, cmap='rocket'):
  heatmap_df = df.corr(numeric_only=True)
  fig, ax = plt.subplots(figsize=(length, width))
  sns.heatmap(data=heatmap_df, annot=True, cmap=cmap, ax=ax)
  # ax.set_title(f'Heatmap of {" and ".join(list(df.columns))}')
  plt.show()

corr_matrix = cleaned_impressions_reach[impression_cols[:2]+reach_cols[:2]+engagement_cols]

# engagements_corr = corr_matrix['Engagements']

# engagements_corr = engagements_corr.sort_values(ascending=False)

# engagements_corr

heatmap(corr_matrix)


In [ ]:
rlc_cols = high_cardinality_columns[21:24]
cleaned_impressions_reach[rlc_cols] = cleaned_impressions_reach[rlc_cols].apply(lambda row: row.str.replace(',','').astype('int'),axis=1)

In [ ]:
corr_matrix = cleaned_impressions_reach[impression_cols[:2]+reach_cols[:2]+engagement_cols+rlc_cols]
heatmap(corr_matrix, length=15, width=10)


In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in rlc_cols:
    df_rt = cleaned_impressions_reach[cleaned_impressions_reach[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

In [ ]:
df = cleaned_impressions_reach[cleaned_impressions_reach['Comments'] > cleaned_impressions_reach['Likes']]

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in rlc_cols:
    df_rt = df[df[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,10))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  df = cleaned_impressions_reach.groupby([col,'Content Type'])[['Content Type']].count().rename(
    columns={"Content Type":"Content Count"}).reset_index()
  sns.barplot(x=col, y='Content Count', hue='Content Type', data=df, ax=axs[i])

  axs[i].set_xlabel(col)
  axs[i].set_ylabel('Count of Content Types')
  axs[i].set_title(f'Trend of Content Types Across {col}')

plt.show()


In [ ]:
click_cols = high_cardinality_columns[24:29]
engage_cols = high_cardinality_columns[29:32]
unique_click_cols = high_cardinality_columns[32:36]

In [ ]:
cleaned_impressions_reach[click_cols].head(5)

In [ ]:
cleaned_impressions_reach[click_cols[3:]] = cleaned_impressions_reach[click_cols[3:]].fillna("0")

In [ ]:
cleaned_impressions_reach[click_cols[1:]] = cleaned_impressions_reach[click_cols[1:]].apply(
    lambda row: row.str.replace(',','').astype('int'), axis=1
)

In [ ]:
opc_pca = cleaned_impressions_reach[cleaned_impressions_reach['Other Post Clicks'] == cleaned_impressions_reach['Post Clicks (All)']]
opc_pca[click_cols+['Content Type']].head(3)
opc_pca[click_cols+engagement_cols].head(3)
opc_pca[click_cols+['Engagements']+rlc_cols].head(3)

In terms of Facebook metrics, when the **Other Post Clicks** count equals the **Post Clicks (All)** count, and both **Post Photo View Clicks** and **Post Video Play Clicks** are zero, it means that users are interacting with your post, but not with its main content. The interactions are likely on elements categorized as "Other", such as clicks on people’s names in comments, clicks on the like count, or clicks on the time.

The **Click-Through Rate (CTR)** being 0% indicates that although users are seeing your post (it's getting impressions), they are not clicking on the links in your post. This could be because the content of the post is not enticing enough to make users want to learn more, or the links are not relevant to the audience.

In terms of how well your post is doing, a high number of **Other Post Clicks** could mean that your post is generating interest and prompting users to interact with it. However, the lack of link clicks and photo/video views suggests that users may not be engaging deeply with your content. They might be reading and interacting with the text and comments, but they're not taking the next step to click on links or view photos/videos.

To improve engagement with your posts, you might want to consider making your content more engaging or relevant to your audience. This could involve using more compelling visuals, crafting more engaging text, or sharing links that are highly relevant to your audience. You could also experiment with different types of content to see what resonates most with your audience.

In [ ]:
opc_pca = cleaned_impressions_reach[cleaned_impressions_reach['Other Post Clicks'] < cleaned_impressions_reach['Post Clicks (All)']]
opc_pca[click_cols+['Content Type']].head(3)
opc_pca[click_cols+engagement_cols].head(3)
opc_pca[click_cols+['Engagements']+rlc_cols].head(3)

In [ ]:
heatmap(cleaned_impressions_reach[engagement_cols+rlc_cols+click_cols], length=15, width=10)

In [ ]:
cleaned_impressions_reach[engage_cols] = cleaned_impressions_reach[engage_cols].apply(
    lambda row: row.str.replace(',','').astype('int'), axis=1
)

In [ ]:
cleaned_impressions_reach[engage_cols].head(5)

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in engage_cols:
    df_rt = cleaned_impressions_reach[cleaned_impressions_reach[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

Comparing the Different Plots, when put alongside the Trend of Reactions, Likes, and Comments, for th Year Column we can see that 2017 and 2020 were the years with peak Reactions, likes, with comments having a peak at 2020 and 2022, but the trends of Engaged Users, Fans and Users talking about the post, have their only peaks at the year 2020, shouldn't more Engaged Users equal more Reactions? Likes and Comments? A sharp drop from the number of Engaged Users, Fans from Afternoon to Evening, Compared to the drop of Reactions, Likes and Comments, Are there Engaged Users but no Reactions, Likes or Comments? The Highest Reactions, and Likes come from Wednesday and Thursday, but Engaged Users, Fans only arrive on Thursday why is that? During the Quarters, the Highest Reactions, Likes, come from the 2nd Quarter with the lowest being in the 3rd Quarter, but here it shows the Engaged Users/Fans dropping from q1 - q4

Trends of Engaged Users/Fans doesn't seem to follow the Trend of Engagements, why is that?

In [ ]:
heatmap(cleaned_impressions_reach[engagement_cols+rlc_cols+click_cols+engage_cols], length=15, width=10)

As we can see there is a very close correlation with Engaged Users/Engaged Fans and Other Post Clicks, Post Clicks (All) and Post Photo View Clicks,we can come to the conclusion that our Engaged Users/ Engaged Fans are those which add to the Other Post Clicks and Post Clicks (All), those Users/ Fans who are clicking links, opening photos, and making comments

In [ ]:
cleaned_impressions_reach[unique_click_cols] = cleaned_impressions_reach[unique_click_cols].fillna("0")
cleaned_impressions_reach[unique_click_cols] = cleaned_impressions_reach[unique_click_cols].apply(
    lambda row: row.str.replace(',','').astype('int'), axis=1
)

In [ ]:
cleaned_impressions_reach[unique_click_cols].head(5)

In [ ]:
heatmap(cleaned_impressions_reach[unique_click_cols+rlc_cols+engage_cols], length=15, width=10)

We see a closer correlation of Engaged Users, Engaged Fans and Users Talking About This, Users Talking About This --> Unique Reactions , Engaged Users/Fans --> Unique Post Clicks/Unique Other Post Clicks.

In [ ]:
video_views_cols = high_cardinality_columns[36:42]
video_views_2_cols = high_cardinality_columns[42:45]
video_views_3_cols = high_cardinality_columns[45:50]
autoplay_views_cols = high_cardinality_columns[51:57]
unique_video_views_cols = high_cardinality_columns[57:]

In [ ]:
dropped_video_data = cleaned_impressions_reach.drop(high_cardinality_columns[36:],axis=1)

### Medium

In [ ]:
medium_cardinality_columns

In [ ]:
dropped_video_data['Sent by'] = np.where(
    dropped_video_data['Sent by']==' ', dropped_video_data['Sent by'].replace(
        ' ','Unknown'), dropped_video_data['Sent by'])

In [ ]:
df_grouped = dropped_video_data.groupby('Sent by')[['Post']].count().sort_values(by=['Post'],ascending=False)
df_grouped

In [ ]:
df_grouped = dropped_video_data.groupby('Sent by')[rlc_cols].sum().sort_values(by=['Reactions'],ascending=False)
df_grouped

In [ ]:
df_grouped = dropped_video_data.groupby('Sent by')[engage_cols].sum().sort_values(by=['Engaged Users'],ascending=False)
df_grouped

In [ ]:
df_grouped = dropped_video_data.groupby('Sent by')[unique_click_cols].sum().sort_values(by=['Unique Post Clicks'],ascending=False)
df_grouped

In [ ]:
df_grouped = dropped_video_data.groupby('Sent by')[reach_cols].sum().sort_values(by=['Reach'],ascending=False)
df_grouped

### Numerical Columns

In [ ]:
num_cols = fb_data.select_dtypes(include=['float64']).columns

In [ ]:
cols_missing = dropped_video_data[num_cols].isnull().sum().to_dict()
for col in cols_missing:
  if cols_missing[col] > 0.5 * dropped_video_data.shape[0]:
    num_cols=num_cols.drop(col)

In [ ]:
impress_reach_cols = list(num_cols[:5])
reactions_cols = list(num_cols[5:10])
sfc = list(num_cols[10:])

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in reactions_cols:
    df_rt = dropped_video_data[cleaned_impressions_reach[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20,15))
axs = axs.ravel()

plot_cols = ['year','time_period','day_name','day_period','quarter','holiday_types']

for i, col in enumerate(plot_cols):
  for column in sfc:
    df_rt = dropped_video_data[cleaned_impressions_reach[col]!='Regular Type']
    df_yearly = df_rt.groupby(col)[column].sum()  # Sum column values for each year
    axs[i].plot(df_yearly.index, df_yearly.values, marker='o', label=column)

    axs[i].set_xlabel(col)
    axs[i].set_ylabel('Count')
    axs[i].set_title(f'Trend of Columns Across {col}')
    axs[i].legend()

plt.show()

In [ ]:
heatmap(dropped_video_data[reactions_cols+rlc_cols+engagement_cols])

### Posts

In [ ]:
dropped_video_data[['Post']].head(5)

In [ ]:
import nltk
from IPython.display import display
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from collections import Counter
nltk.download('stopwords')
nltk.download('vader_lexicon')

import string
import re

#### Preprocessing

In [ ]:
remove_url=lambda x:re.sub(r'http\S+','',str(x))
to_lower=lambda x: x.lower()
remove_puncs= lambda x:x.translate(str.maketrans('','',string.punctuation))

more_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him",
            "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
            "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
            "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as",
            "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before",
            "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then",
            "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some",
            "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don",
            "should", "now"]

stop_words=set(stopwords.words('english')) #nltk package
stop_words.update(more_words)

remove_words=lambda x: ' '.join([word for word in x.split() if word not in stop_words]) #.join is from package string

def preprocess_text(texts):
  texts = texts.apply(remove_url)
  texts = texts.apply(to_lower)
  texts = texts.apply(remove_puncs)
  texts = texts.apply(remove_words)
  return texts

def clean_text(text):
    '''remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

# function to remove emoticons, symbols or flags by their codes
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


In [ ]:
cleaned_posts = preprocess_text(dropped_video_data['Post'])
cleaned_posts = cleaned_posts.apply(lambda x: clean_text(x))
cleaned_posts = cleaned_posts.apply(lambda x: remove_emoji(x))

In [ ]:
filtered_data = dropped_video_data.copy()
filtered_data['Post'] = cleaned_posts

In [ ]:
words_list=[word for line in cleaned_posts for word in line.split()]

# creating dataframe and bar graph of most common 50 words with their frequency
word_counts=Counter(words_list).most_common(50)
word_df=pd.DataFrame(word_counts)
word_df.columns=['word','frq']
display(word_df.head(5))

fig = plt.figure(figsize = (15, 7))

# creating the bar plot
plt.bar(word_df['word'],word_df['frq'])
plt.xticks(rotation=90)
plt.xlabel('word')
plt.ylabel('frq')
plt.title('Most common words')
plt.show()

We can see the Top 50 words with the most frequency in our dataset, words such as itcanbe which is a hashtag, get, stanbic, ibtc, us, visit, email, call, click and so on

In [ ]:
from wordcloud import WordCloud

In [ ]:
cut_text = " ".join(filtered_data['Post'])
max_words=100
word_cloud = WordCloud(
                    background_color='white',
                    stopwords=set(stop_words),
                    max_words=max_words,
                    max_font_size=30,
                    scale=5,
                    colormap='magma',
                    random_state=1).generate(cut_text)
fig = plt.figure(1, figsize=(10,10))
plt.axis('off')
plt.title('Word Cloud for Top '+str(max_words)+' words from Facebook Posts\n', fontsize=10,color='blue')
fig.subplots_adjust(top=2.3)
plt.imshow(word_cloud)
plt.show()

Annotations/Sentiment Analysis

In [ ]:
sid=SentimentIntensityAnalyzer()
ps=lambda x:sid.polarity_scores(x)
sentiment_scores=filtered_data['Post'].apply(ps)

In [ ]:
# create the data frame of negative, neutral, positive and compound polarity scores
sentiment_df=pd.DataFrame(data=list(sentiment_scores))
labelize=lambda x:'neutral' if x==0 else('positive' if x>0 else 'negative')
sentiment_df['sentiment_label']=sentiment_df.compound.apply(labelize)

filtered_data = filtered_data.join(sentiment_df['sentiment_label'])

In [ ]:
filtered_data['sentiment_label'].value_counts().plot(kind='barh',title="Bar Plot of Posts Sentiments");

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
!python -m spacy download en_core_web_lg

In [ ]:
def topic_modelling(text, n=10):

  cvectorizer = CountVectorizer(max_df=0.95, min_df=2,stop_words='english',decode_error='ignore')
  lda_model = LatentDirichletAllocation(n_components=8,learning_method='online',max_iter=20,random_state=42)
  cvz = cvectorizer.fit_transform(text)
  X_topics = lda_model.fit_transform(cvz)
  n_top_words = n
  topic_summaries = []

  topic_word = lda_model.components_  # get the topic words
  vocab = cvectorizer.get_feature_names_out()

  for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

  return cvectorizer, lda_model, topic_summaries

In [ ]:
## Get LDA Topics

vectorizer, lda_model, topic_summaries = topic_modelling(filtered_data['Post'], n=15)

We can create 8 unique categories from this topics

1. **Topic 0 - Future Planning**: This topic seems to be about planning for the future, with keywords like "future", "education", "children", and "time".

2. **Topic 1 - Personal Finance**: This topic appears to be about personal finance and insurance, with keywords like "wealthwednesday", "insurance", "share", and "gift".

3. **Topic 2 - Investment**: This topic seems to be about investment and savings, with keywords like "email", "invest", "account", and "savings".

4. **Topic 3 - Banking Services**: This topic appears to be about banking services, with keywords like "stanbic", "ibtc", "app", and "pension".

5. **Topic 4 - Events & Promotions**: This topic seems to be about events and promotions, with keywords like "join", "win", and "register".

6. **Topic 5 - Business & Economy**: This topic appears to be about business and the economy, with keywords like "business", "growth", and "nigeria".

7. **Topic 6 - Life & Well-being**: This topic seems to be about life and well-being, with keywords like "year", "insurance", and "week".

8. **Topic 7 - Sustainability**: This topic appears to be about sustainability, with keywords like "sdg" (Sustainable Development Goals), "sustainable", and "sustainabilitysaturday".


In [ ]:
topic_labels = ['Future Planning','Personal Finance','Investment','Banking Services','Events & Promotions',
                  'Business & Economy','Life & Well-being','Sustainability']

def assign_topic_to_text(text, cvectorizer, lda_model, topic_labels):
    transformed_text = cvectorizer.transform([text])
    topic_distribution = lda_model.transform(transformed_text)
    best_topic = np.argmax(topic_distribution[0])

    return topic_labels[best_topic]

filtered_data['topic_label'] = filtered_data['Post'].apply(
    lambda x: assign_topic_to_text(x, vectorizer, lda_model, topic_labels))

In [ ]:
nlp = spacy.load('en_core_web_lg')

In [ ]:
def named_entity_recognition(text):
  doc = nlp(text)
  label = [(X.label_) for X in doc.ents]
  return label

def part_of_speech_tagging(text):
  doc = nlp(text)
  label = [(X.pos_) for X in doc]
  return label

filtered_data['ner_label'] = filtered_data['Post'].apply(lambda x:named_entity_recognition(x))
filtered_data['pos_label'] = filtered_data['Post'].apply(lambda x:part_of_speech_tagging(x))

In [ ]:
filtered_data['ner_count'] = filtered_data['ner_label'].apply(lambda x:len(x))
filtered_data['pos_count'] = filtered_data['pos_label'].apply(lambda x:len(x))

In [ ]:
ner_tags = ['CARDINAL','DATE','GPE','LOC','MONEY','ORDINAL','ORG','PERSON','TIME']
for tag in ner_tags:
  filtered_data[tag + '_count'] = filtered_data['ner_label'].apply(lambda x: x.count(tag))

In [ ]:
pos_tags = ['VERB', 'ADV', 'ADJ', 'NUM', 'NOUN', 'SPACE', 'PROPN']
for tag in pos_tags:
    filtered_data[tag + '_count'] = filtered_data['pos_label'].apply(lambda x: x.count(tag))

### Let's Delve in for Deeper Analysis

Let's see the distribution of words for each year

In [ ]:
def get_top_n_words(cleaned_posts, ax, n:int=20):
    words_list=[word for line in cleaned_posts for word in line.split()]
    word_counts=Counter(words_list).most_common(n)
    word_df=pd.DataFrame(word_counts)
    word_df.columns=['word','frq']

    ax.bar(word_df['word'],word_df['frq'])
    ax.set_xticklabels(word_df['word'], rotation=90)
    ax.set_xlabel('word')
    ax.set_ylabel('frq')

def word_cloud(texts, ax, n:int=100):
    cut_text = " ".join(texts)
    max_words=n
    word_cloud = WordCloud(
                      background_color='white',
                      stopwords=set(stop_words),
                      max_words=max_words,
                      max_font_size=30,
                      scale=1,
                      colormap='magma',
                      random_state=42).generate(cut_text)
    ax.axis('off')
    ax.imshow(word_cloud)

def sentiment_analyzer(text, ax):
    sid=SentimentIntensityAnalyzer()
    ps=lambda x:sid.polarity_scores(x)
    sentiment_scores=text.apply(ps)

    sentiment_df=pd.DataFrame(data=list(sentiment_scores))
    labelize=lambda x:'neutral' if x==0 else('positive' if x>0 else 'negative')
    sentiment_df['label']=sentiment_df.compound.apply(labelize)

    sentiment_df['label'].value_counts().plot(kind='barh', ax=ax);

In [ ]:
def visualize_data(df, column, column_value):
    df_year = df[df[column] == column_value]

    fig, axs = plt.subplots(2, 2, figsize=(15, 10))

    get_top_n_words(df_year['Post'].values, axs[0, 0], n=10)
    axs[0, 0].set_title(f"Top 10 Words in the Top 1% Impression {column_value}")

    word_cloud(df_year['Post'].values, axs[0, 1], n=50)
    axs[0, 1].set_title(f"Word Cloud for Top 50 words in the Top 1% Impression {column_value} period")

    labels = sentiment_analyzer(df_year['Post'], axs[1, 1])
    axs[1, 1].set_title(f"Sentiment Analyzer of Tweets in the Top 1% Impression {column_value} period")

    plt.tight_layout()
    plt.show()


def plot_pos_labels(df, column, pos_tags, rows=2, cols=3, plot=False):

    pos_cols = [f"{pos}_count" for pos in pos_tags]

    if plot == False:

      display(df.groupby(column)[pos_cols+['pos_count']].sum().sort_values(by=['pos_count'],ascending=False))

    else:
      df_grouped = df.groupby(column)[pos_cols].sum()

      fig, axs = plt.subplots(rows, cols, figsize=(15, 10))
      fig.patch.set_facecolor('black')

      axs = axs.flatten()

      for ax, (year, row) in zip(axs, df_grouped.iterrows()):
          ax.pie(row, labels=row.index, autopct='%1.1f%%', colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'], textprops={'color':'#ffffff'})
          ax.set_title(f'POS Tag Distribution in {year}', color='white')

      plt.tight_layout()
      plt.show()

def plot_ner_labels(df, column, ner_tags, rows=2, cols=3, plot=False):

    ner_cols = [f"{ner}_count" for ner in ner_tags]

    if plot == False:

      display(df.groupby(column)[ner_cols+['ner_count']].sum().sort_values(by=['ner_count'],ascending=False))

    else:
      df_grouped = df.groupby(column)[ner_cols].sum()

      fig, axs = plt.subplots(rows, cols, figsize=(15, 10))
      fig.patch.set_facecolor('black')

      axs = axs.flatten()

      for ax, (year, row) in zip(axs, df_grouped.iterrows()):
          ax.pie(row, labels=row.index, autopct='%1.1f%%', colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'], textprops={'color':'#ffffff'})
          ax.set_title(f'NER Tag Distribution in {year}', color='white')

      plt.tight_layout()
      plt.show()

In [ ]:
def visualize_labels(df, column, column_values, colors, length=15, width=10):

  ncols = len(column_values)

  fig, axs = plt.subplots(1, ncols, figsize=(length, width))

  for i, value in enumerate(column_values):

    df[df[column]==value]['topic_label'].value_counts().plot(kind='barh', color=colors[i], ax=axs[i])
    axs[i].set_title(f'{value} Topic Labels')

  plt.tight_layout()
  plt.show()


In [ ]:
def barplot_labels(data, plotted_cols, colors, rows=2, cols=4, length=25, width=20):

  fig, axs = plt.subplots(nrows=rows, ncols=cols, figsize=(length, width))
  axs = axs.ravel()

  for i, column in enumerate(plotted_cols):
    df_yearly = data.groupby(['topic_label'])[column].mean()
    df_yearly = df_yearly.sort_values(ascending=True)
    axs[i].barh(df_yearly.index, df_yearly.values, color=colors[i])  # Use color corresponding to column

    axs[i].set_xlabel(column)
    axs[i].set_ylabel('Topic Label')
    axs[i].set_title(f'Distribution of Facebook {column} across Topic Labels', fontsize=10)

  plt.tight_layout()
  plt.show()

In [ ]:
def stack_barchart(data, plot_col, length=15, width=10):

  df = data.groupby([plot_col, 'topic_label'])[['topic_label']].count().rename(columns={'topic_label':'topic_count'})
  df = df.reset_index()

  pivot_df = df.pivot(index='topic_label', columns=plot_col, values='topic_count').fillna(0)

  # Plotting
  plt.figure(figsize=(10,7))
  pivot_df.plot(kind='bar', stacked=True, figsize=(10,7))

  plt.xlabel('Topic Label')
  plt.ylabel('Frequency')
  plt.title(f'Distribution of Topic Labels Across Each {plot_col}')
  plt.legend(title=plot_col)

  plt.show()


In [ ]:
colors = ['skyblue', 'olive', 'gold', 'purple', 'red', 'green', 'orange', 'brown']

#### Distribution of Whole Dataset

In [ ]:
filtered_data['topic_label'].value_counts().plot(kind='bar',title="Distribution of Topic Models");

In [ ]:
pos_cols = [f"{pos}_count" for pos in pos_tags]
ner_cols = [f"{ner}_count" for ner in ner_tags]

filtered_data.groupby(['topic_label'])[pos_cols+['pos_count']].mean().sort_values(by=['pos_count'],ascending=False)

In [ ]:
filtered_data.groupby(['topic_label'])[ner_cols+['ner_count']].mean().sort_values(by=['ner_count'],ascending=False)

In [ ]:
barplot_labels(filtered_data, impression_cols, colors,length=20,width=10)

In [ ]:
barplot_labels(filtered_data, reach_cols, colors, rows=2, cols=3,length=15,width=10)

In [ ]:
filtered_data['Engagement Rate (per Impression)'] = filtered_data['Engagements'] / filtered_data['Impressions'] * 100
filtered_data['Engagement Rate (per Reach)'] = filtered_data['Engagements'] / filtered_data['Reach'] * 100

In [ ]:
filtered_data[engagement_cols] = filtered_data[engagement_cols].replace([np.inf, -np.inf, np.nan], 0)


In [ ]:
barplot_labels(filtered_data, engagement_cols, colors, rows=1, cols=3,length=15, width=10)

In [ ]:
barplot_labels(filtered_data, rlc_cols, colors, rows=1, cols=3,length=15, width=10)

In [ ]:
barplot_labels(filtered_data, click_cols[1:4], colors, rows=1, cols=3, length=15, width=10)

In [ ]:
barplot_labels(filtered_data, engage_cols, colors, rows=1, cols=3, length=15, width=10)

In [ ]:
barplot_labels(filtered_data, unique_click_cols, colors, rows=2, cols=2, length=15, width=10)

In [ ]:
barplot_labels(filtered_data, reactions_cols, colors, rows=2, cols=3, length=15, width=10)

In [ ]:
barplot_labels(filtered_data, sfc, colors, rows=2, cols=4, length=25, width=10)

In [ ]:
stack_barchart(filtered_data, "year")

In [ ]:
stack_barchart(filtered_data, "time_period")

In [ ]:
stack_barchart(filtered_data, "day_name")

In [ ]:
stack_barchart(filtered_data, "day_period")

In [ ]:
stack_barchart(filtered_data, "quarter")

In [ ]:
wrt = filtered_data[filtered_data['holiday_types']!='Regular Type']
stack_barchart(wrt, "holiday_types")

#### Top 1% Impressions

In [ ]:
last_quantile = filtered_data['Impressions'].quantile(0.99) #Posts that had Impression in the Top 1%
df = filtered_data[cleaned_impressions_reach['Impressions'] > last_quantile]

visualize_data(df, "year", 2019)

In [ ]:
visualize_data(df, "year", 2018)

In [ ]:
visualize_data(df, "year", 2022)

In [ ]:
visualize_labels(df, "year", [2018, 2019, 2022], colors, length=15, width=10)

In [ ]:
plot_pos_labels(df, "year",  pos_tags, plot=True)

In [ ]:
plot_ner_labels(df, "year",  ner_tags, plot=False)

Our Insights are finally coming together as we can see and have a clearer explanation now, Posts that were in the top 1% Impression, had the highest Verb count, Adverb count, Adjectives count, Noun Count, Using more parts of Speech in our posts helps us to generate more insights, we can also see compared to the others 2019 had the highest number of topics being discussed. When our posts are distributed it helps to generate more insights, the year 2019 also had the most Named Entities Recognized, so we had Important Dates, Organizations mentioned in our Posts which gathered more Impressions

In [ ]:
visualize_data(df, "time_period", "Morning")

In [ ]:
visualize_data(df, "time_period", "Afternoon")

In [ ]:
visualize_data(df, "time_period", "Evening")

In [ ]:
visualize_labels(df, "time_period", ["Morning", "Afternoon", "Evening"], colors, length=15, width=10)

In [ ]:
plot_pos_labels(df, "time_period", pos_tags, rows=2, cols=2, plot=True)

This proves our theory behind the years with the highest Impression as we can see the same occuring here

In [ ]:
visualize_data(df, "day_name", "Thursday")

In [ ]:
visualize_data(df, "day_name", "Friday")

In [ ]:
visualize_data(df, "day_name", "Saturday")

In [ ]:
visualize_data(df, "day_name", "Sunday")

In [ ]:
visualize_labels(df, "day_name", ["Thursday", "Friday", "Saturday", "Sunday"], colors, length=25, width=20)

In [ ]:
plot_pos_labels(df, "day_name", pos_tags, rows=2, cols=3, plot=True)



This proves our theory behind the years with the highest Impression as we can see the same occuring here


In [ ]:
visualize_data(df, "quarter", "Q1")

In [ ]:
visualize_data(df, "quarter", "Q2")

In [ ]:
visualize_labels(df, "quarter", ["Q1","Q2"],colors, length=15, width=10)

#### Fan 1% Impressions

In [ ]:
last_quantile = filtered_data['Fan Impressions'].quantile(0.99) #Posts that had Impression in the Top 1%
df = filtered_data[cleaned_impressions_reach['Fan Impressions'] > last_quantile]

visualize_data(df, "year", 2019)

In [ ]:
visualize_data(df, "year", 2018)

In [ ]:
visualize_data(df, "year", 2022)

In [ ]:
visualize_labels(df, "year", [2018, 2019, 2022],colors, length=15, width=10)

In [ ]:
plot_pos_labels(df, "quarter",  pos_tags)

In [ ]:
visualize_data(df, "day_name", "Tuesday")

In [ ]:
visualize_data(df, "day_name", "Thursday")

In [ ]:
visualize_data(df, "day_name", "Saturday")

In [ ]:
visualize_labels(df, "day_name", ["Tuesday", "Thursday", "Saturday"], colors, length=25, width=20)

In [ ]:
plot_pos_labels(df, "day_name",  pos_tags)

In [ ]:
visualize_data(df, "quarter", "Q1")

In [ ]:
visualize_data(df, "quarter", "Q2")

In [ ]:
visualize_data(df, "quarter", "Q3")

In [ ]:
visualize_labels(df, "quarter", ["Q1", "Q2", "Q3"],colors,  length=25, width=20)

In [ ]:
plot_pos_labels(df, "quarter",  pos_tags)